**FIRST STEPS**

import libraries for data processing

simple function to process the data coming from DesigoCC WSI 

In [1]:
import pandas as pd
import numpy as np
import dateutil

def preprocess(df):
    print("Beginning data processing...")
    df = df.sort_values('Timestamp')
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.set_index(pd.DatetimeIndex(df['Timestamp']))
    df = df.loc[df['QualityGood'] == 1]
    df = df.drop(columns=['Quality', 'Timestamp'])
    print("Data processing finished.")
    return df

**EXAMPLE csv files**
downloaded from DesigoCC WSI. They contain data of the air ventilation system in D18 building component BT1, L04 classrooms

Timestamps: 

- for air pressure difference (Pa), air temperature (°C) and fu (%) (supply + exhaust air sensors): every 5 minutes
- for control system values (binary on/off mode): time-controlled/irregularly
- for L04 outside temperature: every 30 minutes
- for L04 max. roomtemperature: every 5 minutes

In [ ]:
#exhaust air

abluft_pa = pd.read_csv('2019_D18_BT1_L04/D18_BT1_L04_abluft_paDruck_2019.csv', engine='python', encoding='ISO-8859–1', parse_dates=True)
abluft_pa = preprocess(abluft_pa)

abluft_fu = pd.read_csv('2019_D18_BT1_L04/D18_BT1_L04_abluft_FU_2019.csv', engine='python', encoding='ISO-8859–1', parse_dates=True)
abluft_fu = preprocess(abluft_fu)

#supply air

zuluft_pa = pd.read_csv('2019_D18_BT1_L04/D18_BT1_L04_zuluft_paDruck_2019.csv', engine='python', encoding='ISO-8859–1', parse_dates=True)
zuluft_pa = preprocess(zuluft_pa)

zuluft_fu = pd.read_csv('2019_D18_BT1_L04/D18_BT1_L04_zuluft_FU_2019.csv', engine='python', encoding='ISO-8859–1', parse_dates=True)
zuluft_fu = preprocess(zuluft_fu)

**Calculating air flow volume [m3/min]**

The formula below returns the air flow volume in m3/min on the basis of two variables:
    
- air density at 20°C= 1.2
- cross-sectional area of the airvent-pipe [m2] - this is an assumption, because the exact size is not known = 0.2

In [ ]:
#air flow volume in [m3/min] - without *60 it is m3/s, with *3600 it would be m3/h

def airflow(pa_value):
    if (pa_value > 0).any():
        return((0.2*(np.sqrt((2*pa_value)/1.2)))*60)
    
def airflow_hour(pa_value):
    if (pa_value > 0).any():
        return((0.2*(np.sqrt((2*pa_value)/1.2)))*3600)

abluft_pa['airflow'] = airflow(abluft_pa.Value)
zuluft_pa['airflow'] = airflow(zuluft_pa.Value)

abluft_pa['airflow_hour'] = airflow_hour(abluft_pa.Value)
zuluft_pa['airflow_hour'] = airflow_hour(zuluft_pa.Value)

**Visualization of pa_values, air flow volume**

In [ ]:
import plotly.io as pio
from plotly.subplots import make_subplots

pio.templates.default = "plotly_white"


fig = make_subplots(specs=[[{"secondary_y": True}]])

#add data
fig.add_scatter(secondary_y=False, x=abluft_pa.index, y=abluft_pa.Value, mode="markers",
                marker_color="#dc9200", name="L04_Differenzdruck_Abluft")
fig.add_scatter(secondary_y=True, fill='tozeroy', mode='none', x=abluft_pa.index, y=abluft_pa.airflow,
                marker_color="#d49900", opacity= 0.8, name="Calculated AirFlow Rate_Abluft (A=0.2m3)")

fig.add_scatter(secondary_y=False, x=zuluft_pa.index, y=zuluft_pa.Value, mode="markers",
                marker_color="#704c80", name="L04_Differenzdruck_Zuluft")
fig.add_scatter(secondary_y=True, fill='tozeroy', mode='none', x=zuluft_pa.index, y=zuluft_pa.airflow,
                marker_color="#70566e", opacity= 0.8,  name="Calculated AirFlow Rate_Zuluft (A=0.2m3)")



#yaxis + layout
fig.update_yaxes(secondary_y=False,
                 tickfont_color="#666666", title="Δp = Druckänderung (N/m2)")
fig.update_yaxes(secondary_y=True,
                 tickfont_color="#666666", title="volume in m3/min")
fig.update_layout(title_text="Air pressure difference and calculated air flow rate for L04 D18", legend=dict(x=1.1, y=1))


# Add range slider (optional)

#fig.update_layout(
#    xaxis=go.layout.XAxis(
#        rangeselector=dict(
#            buttons=list([
#                dict(count=1,
#                     label="1d",
#                     step="day",
#                     stepmode="backward"),
#                dict(count=1,
#                     label="1m",
#                     step="month",
#                     stepmode="backward"),
#                dict(step="all")
#            ])
#        ),
#        rangeslider=dict(
#            visible=True
#        ),
#        type="date"
#    )
#)

fig.show()

#saving plot
fig.write_html("AirFlowRate_D18_L04_2019.html")